
# Lab 9: Build a Log Aggregator

In this lab, you will create your own log generator, build a command-line utility that scans log files, summarizes their contents, and provides insight into system behavior. Data structures to track log message levels such as `INFO`, `WARNING`, `ERROR`, and `CRITICAL`.

This lab reinforces:
- File I/O
- Pattern recognition (regex)
- Dictionaries and counters
- Functions and modularity
- CLI arguments, logging



## Part 1: Create Log files (20%)
Using the the following example log format below create a **python file** that will log errors In a structured tree format 

You will find examples in the folder called Logs that you can use to build your program.

Remember set of logs should have a varied levels of log entries (`INFO`, `WARNING`, `ERROR`, `CRITICAL`) and tailored message types for different service components.
You must create 5 structured logs here are some examples:

    sqldb
    ui
    frontend.js
    backend.js
    frontend.flask
    backend.flask

You may use chat GPT to create sample outputs NOT THE LOGS. IE:

    System failure
    Database corruption
    Disk failure detected
    Database corruption


In [ ]:

import os
import random
from datetime import datetime, timedelta
from collections import defaultdict

Log_Folder = "Logs"
os.makedirs(Log_Folder, exist_ok=True)

components = [
    "sqldb", "ui", "frontend.js", "backend.js", "frontend.flask", "backend.flask"
]

log_levels = {
    "INFO": ["Request for information"],
    "WARNING": ["Memory overload"],
    "ERROR": ["Service was interrupted"],
    "CRITICAL": ["System is failing"]
}

#now we define our generator function to create log entries

def generator(component, num_entries=10):
    current_time = datatime.now()
    logs = []

    for _ in range(num_entries):
        time_stamp = current_time.strftime("%Y-%m-%d %H:%M:%S,%f")[:-3]
        level = random.choices(list(log_levels.keys()), weights=[0.5, 0.2, 0.2, 0.1])[0]
        message = random.choice(log_levels[level])
        log_entry = f"{time_stamp} | {component} | {level} | {message}"
        logs.append(log_entry)
        current_time += timedelta(seconds=random.randint(1, 10))
    
    return logs  

def log_files():
    for comp in components[:5]:
        logs = generate_logs(comp)
        file_path = os.path.join(Log_Folder, f"{comp.replace('.', '_')}.log")
        with open(file_path, "w") as f:
            f.write("\n".join(logs))
print("Log files generated in the 'Logs' folder.")

def summarize_logs():
    summary = defaultdict(lamda: defaultdict(int))

    for filename in os.listdir(Log_Folder):
        path = os.path.join(Log_Folder, filename)
        with open(path) as file:
            try:
                _, component, level, _ = map(str.strip, line.split('|',3))
                summary[component][level] += 1
            except ValueError:
                continue
    
    print("\nLog Summary (Component Tree Format):\n")
    for comp, levels in summary.items():
        print(f" {comp}")
        for lvl in ["INFO", "WARNING", "ERROR", "CRITICAL"]:
            if lvl in levels:
                print(f" └── {lvl}: {levels[lvl]}")
        print()

if __name__ == "__main__":
    create_log_files()
    summarize_logs()



### Example Log Format

You will work with logs that follow this simplified structure:

```
2025-04-11 23:20:36,913 | my_app | INFO | Request completed
2025-04-11 23:20:36,914 | my_app.utils | ERROR | Unhandled exception
2025-04-11 23:20:36,914 | my_app.utils.db | CRITICAL | Disk failure detected
```


In [ ]:
#result outputs
2025-04-24 12:01:23,124 | frontend.js         | INFO      | Request completed
2025-04-24 12:01:26,457 | backend.flask       | ERROR     | Unhandled exception
2025-04-24 12:01:29,902 | sqldb               | CRITICAL  | Database corruption


## Part 2: Logging the Log File (40%)
    New File
### Part 2a: Read the Log File (see lab 7) (10%)


Write a function to read the contents of a log file into a list of lines. Handle file errors gracefully.

### Part 2b: Parse Log Lines (see code below if you get stuck) (10%)

Use a regular expression to extract:
- Timestamp
- Log name
- Log level
- Message

### Part 2c: Count Log Levels (20%)

Create a function to count how many times each log level appears. Store the results in a dictionary. Then output it as a Json File
You may pick your own format but here is an example. 
```python
{
    "INFO": 
    {
        "Request completed": 42, 
        "Heartbeat OK": 7
    }

    "WARNING":
    {
        ...
    }
}

```


In [ ]:
import os
import re
import json
from collections import defaultdict

log_pattern = re.compile(
    r"(?P<timestamp>\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3}) \| (?P<logname>[^|]+) \| (?P<level>[^|]+) \| (?P<message>.+)"
)

def parse_log_line(line):
    match = log_pattern.match(line.strip())
    return match.groupdict() if match else None

def count_log_levels(log_folder):
    counts = defaultdict(lambda: defaultdict(int))

    for filename in os.listdir(log_folder):
        filepath = os.path.join(log_folder, filename)
        try:
            with open(filepath) as f:
                for line in f:
                    parsed = parse_log_line(line)
                    if parsed:
                        level = parsed['level']
                        message = parsed['message']
                        counts[level][message] += 1
        except Exception as e:
            print(f"Error reading {filepath}: {e}")

    return counts

def json_summary(data, output_file="log_summary.json"):
    with open(output_file, "w") as f:
        json.dump(data, f, indent=4)
        print(f" Summary to {output_file}")

if __name__ == "__main__":
    log_data = count_log_levels("Logs")
    json_summary(log_data)

    




In [ ]:
#result outputs
{
    "INFO": {
        "Request for information": 12,
        "User logged in": 6
    },
    "ERROR": {
        "Service was interrupted": 3
    }
}



## Step 3: Generate Summary Report (40%)
    New File
### Step 3a (20%):
 Develop a function that continuously monitors your JSON file(s) and will print a real-time summary of log activity. It should keep count of the messages grouped by log level (INFO, WARNING, ERROR, CRITICAL) and display only the critical messages. (I.e. If new data comes in the summary will change and a new critical message will be printed)
 - note: do not reprocess the entire file on each update.  

### Step 3a: Use a Matplotlib (Lecture 10) (20%)
Develop a function that continuously monitors your JSON file(s) and will graph in real-time a bar or pie plot of each of the errors.  (a graph for each log level). 
- The graph should show the distribution of log messages by level  (INFO, WARNING, ERROR, CRITICAL)  


### Critical notes:
- Your code mus use Daemon Threads (Lecture 14)
- 3a and 3b do not need to run at the same time. 


In [ ]:
import json
import time
import os

def monitor_json__summary(json_path):
    last_seen_data = {}

   while True:
        if not os.path.exists(json_path):
            print("Waiting for JSON file")
            time.sleep(1)
            continue

        with open(json_path, 'r') as f:
            try:
                current_data = json.load(f)
            except json.JSONDecodeError:
                print("JSON file not ready...")
                time.sleep(1)
                continue

print("\n Log Level Summary:")
for level, messages in current_data.items():
            total = sum(messages.values())
            print(f"  {level}: {total} messages")


In [ ]:
import matplotlib.pyplot as plt

def live_log_plot(json_path):
    plt.ion()
    fig, ax = plt.subplots()

    while True:
        try:
            with open(json_path, 'r') as f:
                data = json.load(f)

            levels = list(data.keys())
            counts = [sum(data[level].values()) for level in levels]

            ax.clear()
            ax.bar(levels, counts, color="skyblue")
            ax.set_title("Log Message Distribution")
            ax.set_ylabel("Count")
            ax.set_xlabel("Log Level")
            plt.draw()
            plt.pause(2)

        except Exception as e:
            print("Error reading or plotting data:",e)
            time.sleep(2)

if __name__ == "__main__":
    from threading import Thread

    summary_thread = Thread(target=monitor_json_summary, args=("log_summrary.json",))
    plot_thread = Thread(target=live_log_plot, args("log_summary.json",))

    summary_thread.start()
    plot_thread.start()

    summary_thread,join()
    plot_thread.join()

    

    

In [ ]:
# Here is a sample regex that parses a log file and extracts relevant information. 
# you will need to modify it. Review Lecture 11
import re

def parse_log_line(line):
    pattern = r"^(.*?)\s\|\s(\w+)\s\|\s(\w+)\s\|\s(.*)$"
    match = re.match(pattern, line)
    if match: 
        timestamp, log_name, log_level, message = match.groups()
        return {
            "timestamp": timestamp,
            "log_name": log_name,
            "log_level": log_level,
            "message": message
        }
    else:
        return None

def process_log_file(filepath):
    log_summary = {}

    try:
        with open(filepath, 'r') as file:
            for line in file:
                parsed = parse_log_line(line.strip())
                if parsed:
                    level = parsed["log_level"]
                    message = parsed["message"]

                    if level not in log_summary:
                        log_summary[level] = {}
                    if message not in log_summary[level]:
                        log_summary[level][messsage] = 1
                    else:
                        log_summary[level][message] += 1
    except FileNotFoundError:
    print(f"Error: File {filepath} not found.")
    except Exception as e:
    print(f"An error occurred: {e}")

    return log_summary

def save_summary_to_json(summary, output_path):
    with open(output_path, 'w') as json_file:
        json.dump(summary, json_file, indent=4)

if __name__ == "__main__":
    logfile: = "your_log_file.log"
json_output = "log_summary.json"

summary = process_log_file(logfile)
save_summary_to_json(summary, json_output)

print("Finished saving")
   
